# Homework 2 Part 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [2]:
train = pd.read_csv('train.csv')

def process(train):
    ## Your code goes here
    
    train = train.dropna() # Let's just get rid of the NAs.
# Calculate the mean and standard deviation
    mu = np.mean(train['fixed acidity'])
    std = np.std(train['fixed acidity'])
# Normalize the data
    normalized_data = (train['fixed acidity'] - mu)/std
# Find the data index that is 3 standard deviation away from the distribution
    indexes = normalized_data < 3
    train_cleaned = train.loc[indexes, :]
    return train_cleaned

b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [3]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1 - r_squared)
    return vif

c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [4]:
processed_train = process(train) # Getting the cleaned data

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    vif_scores = []
    var = []
    
    new_data = processed_train.drop(['quality'], axis = 1)
    
    for i in new_data.columns:
        
        var = [i]
        x = new_data.drop(i,axis=1).values
        y = new_data[i].values
        
        #calculate regression
        s = sm.add_constant(x)
        results = sm.OLS(y,s).fit()
        r2 = results.rsquared
        
        vif = round(calculate_vif(r2),3)
        var.append(vif)
        vif_scores.append(var)
        
        vif_dataframe = pd.DataFrame(vif_scores, columns = ['Variable', 'VIF'])   
    
    return vif_dataframe

d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [6]:
## Your code goes here
vif = generate_vif_dataframe(processed_train)
vif

,Variable,VIF
0,Id,1.015
1,fixed acidity,7.723
2,volatile acidity,1.783
3,citric acid,3.160
4,residual sugar,1.703
5,chlorides,1.516
6,free sulfur dioxide,1.973
7,total sulfur dioxide,2.183
8,density,6.154
9,pH,3.389


In [6]:
## Your discussion goes here

# According to my results, 'fixed acidity' is the largest VIF, which implies any changes to this variable could result
# in a larger variance in the outputs compared to the other explanatory variables.

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [9]:
## Your code goes here

subset = processed_train.drop(['fixed acidity'],axis = 1).drop(['density'],axis=1).drop(['Id'],axis = 1)

b. Fit a multiple linear regression model and print a summary of the regression result.

In [10]:
## Your code goes here
x = subset.drop('quality',axis = 1).values
y = subset['quality'].values

s = sm.add_constant(x)
prediction = sm.OLS(y,s).fit()
coeff = prediction.params

#print(prediction.summary())

In [11]:
# Code to append stuff to an empty data frame using the indices
newtrain = subset[:0]
ind = np.array([0,23,45,56,68,69,81,98,99])

for i in ind:
    row = subset.iloc[i]
    newtrain = newtrain.append(row)

In [12]:
newtrain

,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,pH,sulphates,alcohol,quality
0,0.370,0.76,4.20,0.066,7.0,38.0,3.22,0.60,13.0,7.0
23,0.575,0.61,2.60,0.076,11.0,24.0,3.16,0.69,9.0,5.0
45,0.820,0.05,2.40,0.081,26.0,96.0,3.36,0.53,10.0,5.0
56,0.490,0.51,2.00,0.422,16.0,62.0,3.03,1.17,9.0,5.0
69,0.370,0.41,2.30,0.088,6.0,16.0,3.09,0.80,9.3,5.0
70,0.600,0.50,2.30,0.079,28.0,71.0,3.50,0.57,9.7,5.0
82,0.735,0.05,2.00,0.081,13.0,54.0,3.39,0.57,9.8,5.0
99,0.490,0.49,2.20,0.083,5.0,15.0,3.19,0.43,9.2,5.0
100,0.700,0.08,3.75,0.067,8.0,16.0,3.43,0.52,12.6,5.0


c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [49]:
## Your code goes here

# kfold function will split the data "x" into "k" equal-sized data sets
def kfold(x,k):
    i = 0 
    while i < k:
        xtrain = np.split(x,k)
        return xtrain
        i += 1

# Linreg function performs a multi-linear regression thru the x data sets & return the coefficients
def linreg(x,y,x2):
    s = sm.add_constant(x)
    r = sm.OLS(y,s).fit()
    x2 = sm.add_constant(x2)
    ypred = r.predict(x2)
    return ypred

# Using all the data except for the last set as my training set

# Calculating the parameters for y estimations
# Order of parameters: constant, sugar, chlorides, free sulfur dioxide

# MSE = (1/n) * sum[(yi - yhat)^2]

def mse(yi,yhat):
    n = len(yi)
    mse_score =  np.sum(np.square(yi - yhat))/n
    return mse_score

# Calculating the MSE between the actual quality given & the estimated quality from multi-linear regression

In [50]:
x1,x2,x3,x4,x5 = kfold(x,5)
y1,y2,y3,y4,y5 = kfold(y,5)

# xtrain1 = pd.concat([x2,x3,x4,x5])
# xtest1 = x1
# ytrain1 = pd.concat([y2,y3,y4,y5])
# ytest1 = y1

# xtrain2 = pd.concat([x1,x3,x4,x5])
# xtest2 = x2
# ytrain2 = pd.concat([y1,y3,y4,y5])
# ytest2 = y2

# xtrain3 = pd.concat([x1,x2,x4,x5])
# xtest3 = x3
# ytrain3 = pd.concat([y1,y2,y4,y5])
# ytest3 = y3

# xtrain4 = pd.concat([x1,x2,x3,x5])
# xtest4 = x4
# ytrain4 = pd.concat([y1,y2,y3,y5])
# ytest4 = y4

# xtrain5 = pd.concat([x1,x2,x3,x4])
# xtest5 = x5
# ytrain5 = pd.concat([y1,y2,y3,y4])
# ytest5 = y5

In [51]:
def main(xtrain,ytrain,xtest):
    pred = linreg(xtrain,ytrain,xtest)
    mse_val = mse(pred,ytrain)
    return mse_val

def avg_mse(mse1,mse2,mse3,mse4,mse5):
    final_mse = round((mse1 + mse2 + mse3 + mse4 + mse5)/5,4)
    print("The average MSE is",final_mse)

In [41]:
mse1 = main(x1,y1)
mse2 = main(x2,y2)
mse3 = main(x3,y3)
mse4 = main(x4,y4)
mse5 = main(x5,y5)
avg_mse(mse1,mse2,mse3,mse4,mse5)

The average MSE is 0.3926


In [47]:
# Use this code!
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
mses = []

for train, test in kf.split(x):
    constant_added = sm.add_constant(x[train,:])
    results = sm.OLS(y[train], constant_added).fit()
    y_pred = results.predict(sm.add_constant(x[test,:]))
    mses.append(mse(y[test], y_pred))
np.mean(mses)

0.4209184549978386

d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [56]:
## Your code goes here

test = pd.read_csv('test.csv', index_col = 0)

x2test = test[['volatile acidity','chlorides','sulphates']]

x2train = processed_train[['volatile acidity','chlorides','sulphates']]
quality = processed_train['quality']

pred = linreg(x2train,quality,x2test)

test['predicted'] = pred
#test.to_csv(r"C:\Users\Duker\Desktop\CS530\Homework\HW2\attempt5.csv")

e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [15]:
# Cross validation result from test data set:
# First Kaggle score is 0.6429
# First MSE score is 0.6185

In [16]:
## Your discussion here

# My MSE score on the training data set is relatively close to the Kaggle score so I think the cross-validation result
# is a good representation of the accuracy. 

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [17]:
## Your discussion here

# According to the MSE's from each set, the test set seems to fit a lot better than the training & c-v set.
# This is simply could be because the test data contains much less data points compared to the other 2.
# However, with the cross-validation method & selecting necessary variables with low VIF scores, I was able to avoid the 
# issues of under-fitting & over-fitting.